In [ ]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Shape of train, val and test should be (nr_samples, 23, 256)
# Every sample of data represents 1 second taken from 23 channels with 256-bit rate
        
npz_file = np.load('/kaggle/input/chbmitseizuredataset/eeg-seizure_train.npz', allow_pickle=True)
signals_train = npz_file['train_signals']
labels_train = npz_file['train_labels']

print('Train data shape is {}'.format(signals_train.shape))
print('Labels shape for train is {}'.format(labels_train.shape))

npz_file = np.load('/kaggle/input/chbmitseizuredataset/eeg-seizure_val.npz', allow_pickle=True)
signals_val = npz_file['val_signals']
labels_val = npz_file['val_labels']


print('Validation data shape is {}'.format(signals_val.shape))
print('Labels shape for validation is {}'.format(labels_val.shape))

npz_file = np.load('/kaggle/input/chbmitseizuredataset/eeg-seizure_test.npz', allow_pickle=True)
signals_test = npz_file['test_signals']


print('Test data shape is {}'.format(signals_val.shape))

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X = np.concatenate((signals_train, signals_val), axis=0)
y = np.concatenate((labels_train, labels_val), axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

X_train = np.transpose(X_train, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
model = models.Sequential([
    layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(256, 23)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
predictions = model.predict(X_test)
print(predictions)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

In [ ]:
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))

In [ ]:
cm = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test[:, 1], predictions[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

# Print some example predictions
print("\nExample Predictions:")
for i in range(10):  # Print first 10 predictions
    print(f"True label: {true_labels[i]}, Predicted probabilities: {predictions[i]}, Predicted label: {predicted_labels[i]}")
